P($m_{th}$ experiment gives significant result | $m$ experiments lacking power to reject $H_0$)

= $(1-\alpha)^{m-1}\alpha$

P(at least one significant result | $m$ experiments lacking power to reject $H_0$)

= $\alpha + (1-\alpha)\alpha + (1-\alpha)^2\alpha + ... + (1-\alpha)^{m-1}\alpha$

= $\frac{\alpha(1 - (1 - \alpha)^m)}{1 - (1 - \alpha)}$

= $1 - (1 - \alpha)^m$

In [106]:
from collections import Counter 
import numpy as np
NUM_SAMPLE = 5000
SAMPLE_LENGTH = 5
GMAX = 3

# randomly generate sample
P = np.random.randint(GMAX, size=(NUM_SAMPLE, SAMPLE_LENGTH))
E = np.random.randint(GMAX, size=(NUM_SAMPLE, SAMPLE_LENGTH))

In [107]:
# TEST UNIT
print('----------------TEST UNIT--------------------\n')
print('Display ranking of P and E:\n')

print('P\n', P, '\n\nNumber of ranking of P:', len(P), '\n')
print('E\n', E, '\n\nNumber of ranking of E:', len(E), '\n') 
print('\n--------------END OF TEST UNIT---------------\n')

----------------TEST UNIT--------------------

Display ranking of P and E:

P
 [[1 0 2 2 1]
 [0 0 2 2 2]
 [1 2 2 1 0]
 ..., 
 [0 0 2 0 1]
 [0 0 0 1 2]
 [0 0 1 2 0]] 

Number of ranking of P: 5000 

E
 [[1 1 2 0 2]
 [1 1 0 0 1]
 [1 0 1 0 0]
 ..., 
 [1 1 2 2 0]
 [0 2 1 2 0]
 [1 1 2 2 1]] 

Number of ranking of E: 5000 


--------------END OF TEST UNIT---------------



In [108]:
# function for computing precision
def precision(ranking):
    counter_rank = Counter(ranking)
    precision = (counter_rank[1] + counter_rank[2]) / sum(counter_rank.values())    
    return precision

P_precision = np.array([])
E_precision = np.array([])

for ranking in P:    
    P_precision = np.append(P_precision, precision(ranking))
    
for ranking in E:    
    E_precision = np.append(E_precision, precision(ranking))
    

In [109]:
# TEST UNIT
print('----------------TEST UNIT--------------------\n')
print('Calculate precision for P and E:\n')

print('Precision for P: ', P_precision)
print('Length of precision for P: ', len(P_precision))
print()
print('Precision for E: ', E_precision)
print('Length of precision for E: ', len(E_precision))

print('\n--------------END OF TEST UNIT---------------\n')

----------------TEST UNIT--------------------

Calculate precision for P and E:

Precision for P:  [ 0.8  0.6  0.8 ...,  0.4  0.4  0.4]
Length of precision for P:  5000

Precision for E:  [ 0.8  0.6  0.4 ...,  0.8  0.6  1. ]
Length of precision for E:  5000

--------------END OF TEST UNIT---------------



In [110]:
#NDCG
def NDCG(ranking): 
    DCG_summation = 0
    NDCG_summation = 0    
    
    # sort ranking for calculating NDCG
    sort_ranking = np.sort(ranking)[::-1]
        
    # calculate DCG and NDCG summation
    for i in range(len(ranking)):        
        DCG_summation += (2 ** ranking[i] - 1) / np.log2(1 + (i+1))
        NDCG_summation += (2 ** sort_ranking[i] - 1) / np.log2(1 + (i+1))               
            
    # if NDCG summation is 0.0, DCG will be 0 too. So we can assign NDCG value as 0.
    if NDCG_summation - 0.0 < 0.000001:
        NDCG = 0.0
    else:
        NDCG = DCG_summation / NDCG_summation  
    
    return DCG_summation, NDCG

P_NDCG = np.array([])
E_NDCG = np.array([])

# calculate NDCG
for ranking in P:
    DCG_one, NDCG_one = NDCG(ranking)
    P_NDCG = np.append(P_NDCG, NDCG_one)
    
for ranking in E:
    DCG_one, NDCG_one = NDCG(ranking)
    E_NDCG = np.append(E_NDCG, NDCG_one)
    

In [111]:
# TEST UNIT
print('----------------TEST UNIT--------------------\n')

print('Calculate NDCG for P and E:\n')

print('NDCG for P: ', P_NDCG)
print('Length of NDCG for P: ', len(P_NDCG))
print('Max value of NDCG for P: ', np.max(P_NDCG))

print()

print('NDCG for E: ', E_NDCG)
print('Length of NDCG for E: ', len(E_NDCG))
print('Max value of NDCG for E: ', np.max(E_NDCG))

print('\n--------------END OF TEST UNIT---------------\n')

----------------TEST UNIT--------------------

Calculate NDCG for P and E:

NDCG for P:  [ 0.71759372  0.6182885   0.82828095 ...,  0.51966106  0.4382445
  0.49354567]
Length of NDCG for P:  5000
Max value of NDCG for P:  1.0

NDCG for E:  [ 0.73693026  0.94690243  0.91972079 ...,  0.75950638  0.68328628
  0.77448719]
Length of NDCG for E:  5000
Max value of NDCG for E:  1.0

--------------END OF TEST UNIT---------------



In [112]:
# ERR

# function to calculate R from relevance
def g2R(g):
    global GMAX
    
    if DEBUG:
        print(GMAX, g)
    
    return (2 ** g - 1) / (2 ** GMAX)

# function to calculate ERR
def ERR(rank_array):
    summation = 0
        
    for g in range(len(rank_array)):                
        
        prod = 1        
        for i in range(g - 1):             
            prod *= 1 - g2R(rank_array[i])                        
            
        summation += prod * g2R(rank_array[g]) / (g + 1)                
        
    ERR = summation
    return ERR
            

In [113]:
P_ERR = np.array([])
E_ERR = np.array([])

for ranking in P:
    ERR_one = ERR(ranking)
    P_ERR = np.append(P_ERR, ERR_one)
    
for ranking in E:
    ERR_one = ERR(ranking)
    E_ERR = np.append(E_ERR, ERR_one)

In [114]:
# TEST UNIT
print('----------------TEST UNIT--------------------\n')

print('Calculate ERR for P and E:\n')

print('ERR for P: ', P_ERR)
print('Length of ERR for P: ', len(P_ERR))

print()

print('ERR for E: ', E_ERR)
print('Length of ERR for E: ', len(E_ERR))

print('\n--------------END OF TEST UNIT---------------\n')

----------------TEST UNIT--------------------

Calculate ERR for P and E:

ERR for P:  [ 0.33007812  0.265625    0.43896484 ...,  0.140625    0.10625     0.13541667]
Length of ERR for P:  5000

ERR for E:  [ 0.33276367  0.20664062  0.16145833 ...,  0.36865234  0.28776042
  0.38061523]
Length of ERR for E:  5000

--------------END OF TEST UNIT---------------



In [115]:
# difference of measure
measure_precision = E_precision - P_precision
measure_NDCG = E_NDCG - P_NDCG
measure_ERR = E_ERR - P_ERR

# indices which E outperformed P.
measure_intersect_idx = np.intersect1d(np.intersect1d(np.where(measure_precision > 0)[0], np.where(measure_NDCG > 0)[0]), np.where(measure_ERR > 0)[0])

# P_intersect and E_intersect is for considering only E outperformed P.
# Remove sample which Measure P >= Measure E
P_intersect = P[measure_intersect_idx]
E_intersect = E[measure_intersect_idx]

In [116]:
# TEST UNIT
print('----------------TEST UNIT--------------------\n')
print('measure precision: ', measure_precision)
print('measure NDCG: ', measure_NDCG)
print('measure_ERR: ', measure_ERR)

print()

# indices which E outperformed P.
print('measure intersect idx: ', measure_intersect_idx)

print()

# Display sample which Measure P >= Measure E
print('P intersect: \n', P_intersect)
print('Length of P intersect: ', len(P_intersect))

print()

print('E intersect: \n', E_intersect)
print('Length of E intersect: ', len(E_intersect))

print('\n--------------END OF TEST UNIT---------------\n')

----------------TEST UNIT--------------------

measure precision:  [ 0.   0.  -0.4 ...,  0.4  0.2  0.6]
measure NDCG:  [ 0.01933654  0.32861393  0.09143984 ...,  0.23984532  0.24504179
  0.28094152]
measure_ERR:  [ 0.00268555 -0.05898438 -0.27750651 ...,  0.22802734  0.18151042
  0.24519857]

measure intersect idx:  [   5    8   15 ..., 4997 4998 4999]

P intersect: 
 [[0 2 0 1 0]
 [2 0 0 0 1]
 [0 1 1 1 2]
 ..., 
 [0 0 2 0 1]
 [0 0 0 1 2]
 [0 0 1 2 0]]
Length of P intersect:  1232

E intersect: 
 [[2 1 0 0 1]
 [2 0 1 0 1]
 [1 2 2 2 2]
 ..., 
 [1 1 2 2 0]
 [0 2 1 2 0]
 [1 1 2 2 1]]
Length of E intersect:  1232

--------------END OF TEST UNIT---------------



In [117]:
# Team Draft Interleaving

# interleaved ranking
interleaved_ranking = np.ndarray((len(P), SAMPLE_LENGTH * 2), dtype=int)

# for recording documents in ranking should be credited to P or E
interleaved_PE = np.ndarray((len(P), SAMPLE_LENGTH * 2), dtype=int)

# We assume P, E retrieve different documents and we don't have docID. 
# So return TEAM_P and TEAM_E is not necessary.

for count_sample in range(len(P)):
    count_team_P = 0
    count_team_E = 0    
    
    count_while_idx = -1
    while count_team_P < SAMPLE_LENGTH or count_team_E < SAMPLE_LENGTH:
        count_while_idx += 1
        
        if count_team_P < count_team_E or (count_team_P == count_team_E and np.random.randint(2) == 1):
            P_idx = count_team_P
            interleaved_ranking[count_sample][count_while_idx] = P[count_sample][P_idx]
            count_team_P += 1
            interleaved_PE[count_sample][count_while_idx] = 0
            
        else:
            E_idx = count_team_E
            interleaved_ranking[count_sample][count_while_idx] = E[count_sample][E_idx]
            count_team_E += 1
            interleaved_PE[count_sample][count_while_idx] = 1

In [118]:
# TEST UNIT
print('----------------TEST UNIT--------------------\n')
print('Interleaved ranking: \n', interleaved_ranking)

print()

print('Length of one interleaved ranking: ', len(interleaved_ranking[0]))
print('Length of all interleaved rankings: ', len(interleaved_ranking))
print('\n--------------END OF TEST UNIT---------------\n')

----------------TEST UNIT--------------------

Interleaved ranking: 
 [[1 1 0 ..., 0 2 1]
 [1 0 1 ..., 2 1 2]
 [1 1 0 ..., 0 0 0]
 ..., 
 [1 0 0 ..., 2 1 0]
 [0 0 2 ..., 1 2 0]
 [0 1 0 ..., 2 1 0]]

Length of one interleaved ranking:  10
Length of all interleaved rankings:  5000

--------------END OF TEST UNIT---------------



In [119]:
yandex = open('YandexRelPredChallenge.txt', 'r')

# count number of query and click for calculating RCM
count_Q = 0
count_C = 0

# for sigma_uq of SDBN
count_uq_dict = {}
count_uq_last_dict = {}
sigma_uq = {}

# for converting N, R, HR to sigma_uq
sigma_N = 0
sigma_R = 0
sigma_HR = 0
count_R = 0
count_HR = 0

# for checking if one click is the last click of one query
previous_typeOfAction = None
last_queryID = None
last_URLID = None

# processing yandex for calculating parameters of RCM and SDBN
for line in yandex:
    count += 1
    
    line_split = line.split('\t')
    
    typeOfAction = line_split[2]
    
    if typeOfAction == 'Q':
        
        if previous_typeOfAction == 'C':
            if (last_queryID, last_URLID) in count_uq_last_dict:
                count_uq_last_dict[(last_queryID, last_URLID)] += 1
            else: 
                count_uq_last_dict[(last_queryID, last_URLID)] = 1
                
        count_Q += 1
        
        sessionID = line_split[0]
        timePassed = line_split[1]
        queryID = line_split[3]
        regionID = line_split[4]
        listOfURLs = line_split[5:]
        listOfURLs[-1] = listOfURLs[-1][:-1]
        
        last_queryID = queryID        
        
    elif typeOfAction == 'C':
        count_C += 1
        
        sessionID = line_split[0]
        timePassed = line_split[1]        
        URLID = line_split[3]    
        URLID = URLID[:-1]        
            
        last_URLID = URLID
            
        if (last_queryID, URLID) in count_uq_dict:
            count_uq_dict[(last_queryID, URLID)] += 1
        else: 
            count_uq_dict[(last_queryID, URLID)] = 1
    
    previous_typeOfAction = typeOfAction
    
    
for idx, denominator in count_uq_dict.items():
    
    if idx in count_uq_last_dict:
        nominator = count_uq_last_dict[idx]
    else:
        nominator = 0.0
    
    sigma = nominator / denominator
    
    sigma_uq[idx] = sigma
    
    # define sigma > 0.8 as HR
    if sigma > 0.8:
        #relevance = 2
        sigma_HR = (count_HR * sigma_HR + sigma) / (count_HR + 1)
        count_HR += 1
        
    # define sigma >= 0 as R, at least the document is clicked after reading snippet.
    else:
        #relevance = 1
        sigma_R = (count_R * sigma_R + sigma) / (count_R + 1)
        count_R += 1    
        
    # if index(document-query pair) is not in sigma_uq, relevance will be 0 for model.
    
num_shown_doc = count_Q * 10
RCM_p = count_C / num_shown_doc



In [120]:
# TEST UNIT
print('----------------TEST UNIT--------------------\n')
print('RCM probability: ', RCM_p)

print()

print('sigma for N: ', sigma_N)
print('sigma for R: ', sigma_R)
print('sigma for HR: ', sigma_HR)

print('\n--------------END OF TEST UNIT---------------\n')

----------------TEST UNIT--------------------

RCM probability:  0.13445559411047547

sigma for N:  0
sigma for R:  0.06145912887528477
sigma for HR:  0.999079479392326

--------------END OF TEST UNIT---------------



In [121]:
# convert relevance to sigma for SDBN
def g2Sigma(g):
    global sigma_N, sigma_R, sigma_HR
    if g == 2:
        return sigma_HR
    elif g == 1:
        return sigma_R    
    elif g == 0:
        return sigma_N
    
    raise AssertionError()
    
# RCM    
def RCM(ranking, p):
    clicked_ranking = []    
    
    for idx, relevance in enumerate(ranking):
        if np.random.rand() <= p:
            clicked_ranking += [1]
        else:
            clicked_ranking += [0]
            
    return clicked_ranking
        
# SDBN
def SDBN(ranking):
    global sigma_N, sigma_R, sigma_HR
    clicked_ranking = []
    
    for idx, relevance in enumerate(ranking):        
        
        # SDBN gamma = 1
        P_E = 1
        P_A = g2R(relevance)
        P_C = P_E * P_A
        
        if np.random.rand() <= P_C:
            clicked_ranking += [1]
            P_S = g2Sigma(relevance)
            if np.random.rand() <= P_S:
                if idx < len(ranking) - 1:
                    clicked_ranking += (len(ranking) - 1 - idx) * [0]
                break
        else:
            clicked_ranking += [0]
        
        
        
    return clicked_ranking

In [122]:
# TEST UNIT
print('----------------TEST UNIT--------------------\n')
print('Test click models(1 means clicked): \n')

print('RCM:', RCM(P[0], RCM_p))

print()

print('SDBN', SDBN(P[0]))
print('\n--------------END OF TEST UNIT---------------\n')

----------------TEST UNIT--------------------

Test click models(1 means clicked): 

RCM: [0, 1, 0, 1, 0]

SDBN [0, 0, 0, 1, 0]

--------------END OF TEST UNIT---------------



In [123]:
RCM_P_WIN = 0
RCM_E_WIN = 0
RCM_TIE = 0

SDBN_P_WIN = 0
SDBN_E_WIN = 0
SDBN_TIE = 0

for idx, interleave in enumerate(interleaved_ranking):    
    RCM_simulation = RCM(interleave, RCM_p)
    SDBN_simulation = SDBN(interleave)
    
    RCM_clicked_index = np.where(np.array(RCM_simulation) == 1)
    SDBN_clicked_index = np.where(np.array(SDBN_simulation) == 1)
    
    RCM_winners = interleaved_PE[idx][RCM_clicked_index]
    SDBN_winners = interleaved_PE[idx][SDBN_clicked_index]
    
    RCM_count_P_winner = len(np.where(RCM_winners == 0)[0])
    RCM_count_E_winner = len(np.where(RCM_winners == 1)[0])
    
    SDBN_count_P_winner = len(np.where(SDBN_winners == 0)[0])
    SDBN_count_E_winner = len(np.where(SDBN_winners == 1)[0])
    
    if RCM_count_P_winner > RCM_count_E_winner:
        RCM_P_WIN += 1
    elif RCM_count_P_winner < RCM_count_E_winner:
        RCM_E_WIN += 1
    else:
        RCM_TIE += 1
        
    if SDBN_count_P_winner > SDBN_count_E_winner:
        SDBN_P_WIN += 1
    elif SDBN_count_P_winner < SDBN_count_E_winner:
        SDBN_E_WIN += 1
    else:
        SDBN_TIE += 1


1521 1497 1982 1890 1965 1145


In [125]:
# TEST UNIT
print('----------------TEST UNIT--------------------\n')
print('Simulate results: \n')
print('Count RCM P win: ', RCM_P_WIN)
print('Count RCM E win: ', RCM_E_WIN)
print('Count RCM tie: ', RCM_TIE)
print('Count SDBN P win: ', SDBN_P_WIN)
print('Count SDBN E win: ', SDBN_E_WIN)
print('Count SDBN tie: ', SDBN_TIE)
print('\n--------------END OF TEST UNIT---------------\n')

----------------TEST UNIT--------------------

Simulate results: 

Count RCM P win:  1521
Count RCM E win:  1497
Count RCM tie:  1982
Count SDBN P win:  1890
Count SDBN E win:  1965
Count SDBN tie:  1145

--------------END OF TEST UNIT---------------

